<a href="https://colab.research.google.com/github/emerenan/xlsx_validation/blob/main/es_validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime
import re
from openpyxl import Workbook, styles
from openpyxl.styles import PatternFill, Font
from openpyxl.styles.differential import DifferentialStyle
from openpyxl.formatting.rule import Rule

def read_files(path, sheetname, n_skiprows, n_skip_columns, site_index):
    """
    Params:\n
    path: parth of file in the computer.\n
    n_skiprows: Number of rows to delete in the original file,.\n
    columns_to_convert: Columns to convert the data general type. \n
    n_skipcolumn: Columns to skip in the original file. \n
    endrow = pass 0 to read everything, 1 to count entire
    columns_order: List of columns names in specific order to pass in the engine.\n
    """
    df = pd.read_excel(path, sheet_name = sheetname, skiprows = n_skiprows)

    df = df.iloc[:,n_skip_columns:]

    # define the entiry columns which doesn't have NaN 
    df = df.dropna(subset=[site_index], axis=0)
    """cont = 0
    for i in df.iloc[:,site_col]:
        if pd.isnull(i):
            break # acertar o break
        else:
            cont +=1
    df = df.iloc[:cont, :]
    #df.columns = columns_order"""
    
    # convert intery columns to integer 
    #df[columns_integer_convert] = df[columns_integer_convert].fillna(0)
    #df[columns_integer_convert] = df[columns_integer_convert].astype('int64')

    return df

def lower_str(columns):
    newlist = list(map(lambda x: x.lower(), columns))
    return newlist

def check_df(df, error_msg):
    if df == None or df.empty:
        return f'{error_msg}'
    else:
        return df

def count_duplicates(lista):
    count_dict = {}
    for entry in lista:
        if entry in count_dict.keys():
            count_dict[entry] += 1
        else:
            count_dict[entry] = 1
    
    duplicates = {}
    for k, v in count_dict.items():
        if v > 1:
            duplicates[k] = v
    return pd.DataFrame.from_dict(duplicates, orient='index', columns=['# of Duplicates'])

def defining_df(df, column_range, number_col):
    cont = 0
    for i in df.iloc[:,number_col]:
        if pd.isnull(i):
            break # acertar o break
        else:
            cont +=1
    df = df.iloc[0:cont, :]
    return df

#old version
def check_columns(table, output_columns):
    """
    Check the total of number of missing columns and the missing columns in passed table.\n

    Params:\n
    table: contain the columns to be check.\n
    output_columns: columns structure at the final file. 

    Returns:\n
    Number of missing columns and a list that contains the name os missing columns.
    """
    """"
    countries = ['DE':{'towerDB':[],\
                       'lc': [],\
                       'ta':[]}\,
                 'HU',
                 'IE',
                 'RO',
                 'PT',
                 'ES',
                 'CZ': {'towerDB':["Code (Duplicate)","Site Status","VF - In scope / out of scope (Generalised scoping)","Site in Skylon scope Actual (From Site List Sheet )","Legacy Site Code(Duplicate)","TIMS Site Code","Legacy Site Code","Site Name","Macro Region","Province","Municipality","Inhabitants","Address","Ground Register","Altitude","Latitude","Longitude","Categorization by Inhabitants","Categorization by Transmission Sys","Categorization by Site Type","Categorization by Transmission Sys (subcluster)","Other internal Categorization 1 (Identify ACQ Sites)","Other internal Categorization 2 Energy provider (Eon/ LL)","DAS+Macro","DAS (Yes/ No)","DAS Ownership (Complete/ Partial/ 3rd Party)","Active/ Passive DAS","# of remote units/ radiating points","Type of Structure","Distance highest antenna to ground level","GBT Tower height","POD ID","Energy Consumption LTM (kwh)","Annual Energy cost LTM (Euros)","Infrastructure ready (existing)/ to be ready (new)","Infrastructure to be dismantled by","Radio equipments to be deactivated by","Infrastructure to be shared by","Technology VOD","Fibre / Microwave","Vertical passive structure owner","Room configuration (detailed)","Shelter passive structure ownership","Type of Air Conditioning","Number of cabinets (Full Capacity)","Number of Antenna (Full Capacity)","Number of MW (Full Capacity)","Counterpart","# of Lease Contracts","Current annual lease fees ","Current other fees (Maintenance)","Current other fees","(Average) residual duration - Lease contract","(Average) residual duration - Maintenance contract","(Average) residual duration - Lease & Maintenance both","# of Tenants Agreements","Current Total Annual Hosting Fees","Tenant (name/ID) MNO1 (Česká telekomunikační infrastruktura a.s.)","Annual Fee per Tenant MNO1","Annual Energy Fee MNO1","Annual Maintenance Fee MNO1","Other Services Fee MNO1","Residual duration MNO1 (Years)","Tenant (name/ID) MNO2 (T-Mobile Czech Republic a.s.)","Annual Fee per Tenant MNO2","Annual Energy Fee MNO2","Annual Maintenance Fee MNO2","Other Services Fee MNO2","Residual duration MNO2 (days)","Tenant (name/ID) MNO3","Annual Fee per Tenant MNO3","Annual Energy Fee MNO3","Annual Maintenance Fee MNO3","Other Services Fee MNO3","Residual duration MNO3","# of OTMOs","Annual Fee from OTMOs","Annual Energy Fee from OTMOs","Annual Maintenance Fee OTMOs","Other Services Fee OTMOs","Average residual duration (days)","Check","Strategic Macro Sites","Critical Sites","Case A Core Site","Macro Site - Transmission Hub Site","Macro Site - Transmission Hub Site with/without Shelters","Transmission Sites","Room Configuration","Power Supply","Air Conditioning","Active Sharing Arrangements involving the Operator","VF-CZ Demerger phase","EVO Location [FAR Site ID] ","Billing Trigger date ",\
                 "Strategic_Site_Bucket","Critical Site Bucket","Subsequent_Sharing_Arrangement","First_Active_Sharing_Deployment_Type","First_Active_Sharing_Start_Date","First_Active_Sharing_End_Date","Decommissioned_Site","Wip_Site","Bts_Site","Transfer_Date_Of_Consent_Required_Sites","Sites_As_Metered_Estimated","Date_Of_Equipment_Removal"],\
                       'lc': [],\
                       'ta':['Tenant Agreement ID - NEW', 'Code', 'Site Name', 'Service Type','Contract start date', 'Contract end date', 'Status of contract','Renewal Option', 'Comments', 'Counterpart ID', 'Counterpart','Classification of Tenant', 'Annual amount - billing currency','Billing Currency', 'Annual Amount in CZK', 'Amount in EUR','Terms of Payment', 'Frequency', 'Indexed YES/NO', 'Index','Percentage', 'VAT Subject YES/NO', 'Percentage (VAT)', 'Unique Key','Classification', 'Residual Period in Years', 'Remarks','Count of unique key', 'Count of contracts', 'Scoping classification','DAS sites', 'DAS ownership', '31-Mar-21', 'Update in month','Updated Item', 'Comment', 'Counterpart_extra_1', 'Counterpart_extra_2', 'x','SiteType', '1.028', 'Unique Tenant', 'Unique Tenant Count', 'not_def_1']}\,
                 }
                 'GR']
                 """
    total_received = len(table.columns)
    number_missing_columns = 0
    missing_columns = []
    #Counting of missing columns       
    #if country in contries:      
    for columns in output_columns:
        if columns.lower() not in [labels.lower() for labels in table]:
            number_missing_columns +=1
            missing_columns.append(columns)
    
    return total_received, number_missing_columns, missing_columns

def check_columns_received(df, bill_cols):
    twdb_col = lower_str(list(df.columns))
    col_miss = [i for i in bill_cols if i not in twdb_col]
    """
    for i in bill_cols:
        if i not in twdb_col:
            col_miss.append(i)"""
    df_col_missing = pd.DataFrame(col_miss, columns=['Column(s) Missing'], index=range(len(col_miss)))
    return df_col_missing

def replace_values(df, columns, value=""):
    """
    Está voltando para float
    """
    invalid_values = ['N/A', 'n/a',"0", 0, '-', '_', np.nan,'nan']
    #lista = []

    df.fillna(0, inplace=True)
    #df[column] = df[column].astype('int64')

    for column in columns:
        lista = []
        for index in df[column]:
            #print(f"{column} -> {index}")
            if index in invalid_values:
                lista.append(value)
            else:
                lista.append(index)
        df[column] = lista

    return df
      
def date_parser(df, columns, format, type_dates):
    t_col = type_dates.lower()
    for column in columns:
        if t_col == 'mixed':
            df[column] = [date_obj.strftime(format) if not pd.isnull(date_obj) \
                        and not isinstance(date_obj, str) else date_obj for date_obj in df[column]]
            df[column] = df[column].astype(str)
        else:
            df[column] = [date_obj.strftime(format) if not pd.isnull(date_obj) else '' for date_obj in df[column]]
            df[column] = df[column].astype(str)

# Refactorar esse codigo para receber todas as colunas num dic
# Sendo as keys=columns e values= picklist for each column
def check_date_columns(df, df_index, columns, format):
    """
    Paramns \n
        Dates needs to be in string format not in datetime, otherwise raise an error.\n
        Convert entire column to string before.
    """
    df_dates = df[columns]
    df_dates['sites'] = df_dates[df_index]
    df_dates = df_dates.set_index('sites')

    df_de = pd.DataFrame(columns=columns)
    
    df_duplicates = count_duplicates(df_dates[df_index])
    if not df_duplicates.empty:
        df_dates.drop_duplicates(subset=[df_index], inplace=True)
    
    if format == 1:
        date_format = re.compile(r'\d{2}\-\d{2}\-\d{4}')
        for de_site in df_dates[df_index]:
            for de_column in columns[1:]:
                #match = re.search(r'\d{2}\/\d{2}\/\d{4}', df_dates.loc[ta_site,ta_column])
                #print(type(df_dates.loc[de_site,de_column]))
                if date_format.match(df_dates.loc[de_site,de_column]) == None:
                    if df_dates.loc[de_site,df_index] not in df_de.index:
                        df_de.loc[de_site,df_index] = df_dates.loc[de_site,df_index]
                        df_de.loc[de_site,de_column] = 'Incorret Format or Blank Value'
                    else:
                        df_de.loc[de_site,de_column] = 'Incorret Format or Blank Value'
        df_de = df_de.dropna(how='all', axis=1).fillna('Ok')       
        if not df_de.empty:
            return df_de
        else: 
            print('No one columns with incorrect date format!')
    else:
        date_format = re.compile(r'\d{2}\/\d{2}\/\d{4}')
        for de_site in df_dates[df_index]:
            for de_column in columns[1:]:
                #match = re.search(r'\d{2}\/\d{2}\/\d{4}', df_dates.loc[ta_site,ta_column])
                if date_format.match(df_dates.loc[de_site,de_column]) == None:
                    if df_dates.loc[de_site,df_index] not in df_de.index:
                        df_de.loc[de_site,df_index] = df_dates.loc[de_site,df_index]
                        df_de.loc[de_site,de_column] = 'Incorret Format or Blank Value'
                    else:
                        df_de.loc[de_site,de_column] = 'Incorret Format or Blank Value'
        df_de = df_de.dropna(how='all', axis=1).fillna('Ok')       
        if not df_de.empty:
            return df_de
        else: 
            print('No one columns with incorrect date format!')

def check_amounts(df_check, df_index, columns, pattern=','):
    """
    Paramns:
    pattern: general (. to decimal), other (, to decimal)
    """

    df = df_check[columns]
    df['sites'] = df[df_index]
    df = df.set_index('sites')

    df_new = pd.DataFrame(columns=columns)
    
    df_duplicates = count_duplicates(df[df_index])
    if not df_duplicates.empty:
        df.drop_duplicates(subset=[df_index], inplace=True)

    for site in df[df_index]:
        for column in columns[1:]:
            if not str(df.loc[site,column]).__contains__(pattern):
                #print(df.loc[site,column])
                if df.loc[site,df_index] not in df_new.index:
                    df_new.loc[site,df_index] = df.loc[site,df_index]
                    df_new.loc[site,column] = 'Incorret Format to separate decimal values'
                else:
                    df_new.loc[site,column] = 'Incorret Format to separate decimal values'

    df_new = df_new.dropna(how='all', axis=1).fillna('Ok')       
    if not df_new.empty:
        return df_new
    else: 
        print('No one columns with incorrect Amount format!')
        
def check_picklist(df,df_index,df_cols, picklist_dict):
    """
    Params:
        df:
        df_index: 
        picklist_dict:
        nultiIndex: More than one column to check
    """
    df_picklist = df[df_cols]
    df_picklist['sites'] = df[df_index]
    df_picklist =  df_picklist.set_index('sites')
    
    #df_picklist = replace_values(df_picklist, df_cols, 0)

    df_errors = pd.DataFrame(columns=df_cols)

    for site in df[df_index]:
        for column in picklist_dict.keys(): 
            value = str(df_picklist.loc[site,column])
            col_values = [i.lower() for i in picklist_dict[column]]
            if not value.lower() in col_values or pd.isnull(value):
                if not df_picklist.loc[site,df_index] in df_errors.index:
                    df_errors.loc[site,df_index] = df_picklist.loc[site,df_index]
                    df_errors.loc[site,column] = 'Incorret picklist value or Blank Value'
                else:
                    df_errors.loc[site,column] = 'Incorret picklist value or Blank Value'
    #df = df_errors.dropna()   
    df = df_errors.dropna(how='all', axis=1) 
    
    return df

def check_picklist_3(df,df_index, picklist_dict):
    log = {}
    for column in picklist_dict:
        df_aux = df.copy()
        new = df_aux[column].isin(picklist_dict[column])
        #new = df_aux[column].apply(lambda x: x in picklist_dict[column])
        # Aceita somento os valores que não estão na picklist
        indexes = df.index[new == False].tolist()
        #if len(indexes)>0:
        if column not in log:log[column]=[]
        log[column]=log[column]+indexes
    #print(log.keys())

    newDict ={}
    df1 = pd.DataFrame()
    for key,value in log.items():
        for val in value:  
            ID=df.iloc[val][df_index]
            if ID in newDict:
                newDict[ID].append(key)
            else:
                newDict[ID] = [key]
        
    logs = pd.DataFrame.from_dict(newDict, orient='index')
    return logs

#check on air foi trocado pelo check_picklist
def check_on_air_sites(df, df_index, df_cols):

    df_check = df[df_cols]
    df_check['sites'] = df_check[df_index]
    df_check = df_check.set_index('sites')
    #df_check = replace_values(df_check, df_cols, 0)

    df_errors = pd.DataFrame(columns=df_cols)

    for site in df[df_index]:
        for column in df_cols: 
            if df_check.loc[site,df_index] not in df_errors.index:
                df_errors.loc[site,df_index] = df_check.loc[site,df_index]
                if df_check.loc[site,column] == 0:
                    df_errors.loc[site,column] = 'Incorret picklist value or Blank Value'
            else:
                if df_check.loc[site,column] == 0:
                    df_errors.loc[site,column] = 'Incorret picklist value or Blank Value'
    df = df_errors[df_errors.iloc[:,1:]]
    df = df.dropna(how='all', axis=1).fillna('Ok')       
    #df = df_errors.dropna(how='all', axis=1)   
    return df

def check_new_sites(df_towerdb, tw_index, bts_col, bill_col, msa_list, towerdb_list, uip_list):
    
    # capture current date as string
    current_date = pd.to_datetime('now').date()
    # convert current to timestamp
    current_date = pd.to_datetime(current_date)
    
    #Finding for ALL NEW SITES
    new_site = [str(i) for i in towerdb_list if i not in msa_list]
    new_sites = pd.DataFrame(new_site, columns=['New_Sites'])
    
    #list of new sites out of UIP sheet
    bts_out_uip = [str(i) for i in df_towerdb[df_towerdb[bts_col]=='Yes'][tw_index].sort_values() if i not in uip_list]
    bts_out_uip = pd.DataFrame(bts_out_uip, columns=['Bts_Sites_Out_UIP_File'])

    #create a copy to make index modifications
    df = df_towerdb.copy()

    #New columns with Site Codes
    df['Sites'] = df[tw_index]
    #defining site code to index
    df.set_index('Sites', inplace=True)

    #filtering by new sites
    df = df[df[tw_index].isin(new_site)]

    # Save information os sites with demerged date more than current date
    df[bill_col] = df[bill_col].astype('datetime64[s]')
    df_site_bts = df[(df[bts_col]=='Yes') | (df[bill_col] > current_date)]

    return new_sites, bts_out_uip, df_site_bts[[tw_index, bts_col, bill_col]]
    """Restruturar o script em CZ, DE, PT"""

def check_bts(df_tw, bts_tw_columns, tw_index, df_msa, bts_msa_column, msa_index):
    #Nested Function to make conditional validations
    def cond_bts_check(bts_msa, tw_bts_sites):
        bts_out_tw=[]
        if sorted(bts_msa) != sorted(tw_bts_sites):
            for i in tw_bts_sites:
                if i not in bts_msa:
                    bts_out_tw.append(i)

        return bts_out_tw

    bts_msa = msa[msa[bts_msa_column]=='Yes']
    bts_msa = [str(i) for i in bts_msa[msa_index]]

    tw_bts_sites = df_tw[df_tw[bts_tw_columns]=='Yes']
    tw_bts_sites = [str(i) for i in tw_bts_sites[tw_index]]

    #return of datas
    bts_out_tw = cond_bts_check(bts_msa, tw_bts_sites)
    df = pd.DataFrame(bts_out_tw, columns=['New Sites'])
    return df

def check_wip(df_tw,tw_index, wip_tw, tw_bts, df_msa, msa_index, wip_msa_col):

    #Nested Function to make conditional validations
    def cond_wip_check(wip_msa, tw_wip_sites):
        count_wip = 0
        wip_out_tw=[]
        if sorted(wip_msa) != sorted(tw_wip_sites):
            for i in tw_wip_sites:
                if i not in wip_msa:
                    count_wip += 1
                    wip_out_tw.append(i)

        return wip_out_tw

    wip_msa = df_msa[df_msa[wip_msa_col]=='Yes']
    wip_msa = [str(i) for i in wip_msa[msa_index]]

    tw_wip_sites = df_tw[df_tw[wip_tw]=='Yes']
    tw_wip_sites = [str(i) for i in tw_wip_sites[tw_index]]

    tw_wip_site_bts_flagged = df_tw[(df_tw[wip_tw]=='Yes')&(df_tw[tw_bts]=='Yes')]
    tw_wip_site_bts_flagged = tw_wip_site_bts_flagged[[tw_index, wip_tw, tw_bts]]

    wip_out_tw_list = cond_wip_check(wip_msa, tw_wip_sites)
    return wip_out_tw_list, tw_wip_site_bts_flagged
    """Reestrurar os script em PT, DE, CZ"""
    # Falta os outros países

def check_decommissioned(df,df_index, decom_col, doer_col):
    #c = country.lower()
    filtered = df[(df[decom_col]=='Yes')&(df[doer_col]=="")]
    return filtered[[df_index, decom_col, doer_col]]
    """Ajustar para CZ, DE, PT"""
    
def check_tw_bill_doer(df_tw, tw_index, date_col, status_col, status, type_col):
    
    t = type_col.lower()
    # capture current date as string
    current_date = pd.to_datetime('now').date()
    # convert current to timestamp
    current_date = pd.to_datetime(current_date)
    if not df_tw[date_col].empty:
        if t == 'doer':
            filtered = df_tw[(df_tw[status_col]==status)&(df_tw[date_col].astype('datetime64[ns]') < current_date)]
            return filtered[[tw_index, status_col, date_col]] 
        else:
            filtered = df_tw[(df_tw[status_col]==status)&(df_tw[date_col].astype('datetime64[ns]').empty)]
            return filtered[[tw_index, status_col, date_col]] 
    else:
        print('Nothing wrong in services sites!')
    """Ajustar para CZ, DE, PT, IE"""

def check_tw_doer_planned(df_tw, tw_index, doer_col, status_col):
    """Only GR until now"""
    # capture current date as string
    current_date = pd.to_datetime('now').date()
    # convert current to timestamp
    current_date = pd.to_datetime(current_date)
    if not df_tw[doer_col].empty:
        filtered = df_tw[(df_tw[status_col]=='Planned')&(not df_tw[doer_col].astype('datetime64[ns]').empty)]
        return filtered[[tw_index, status_col, doer_col]]  
    else:
        print('Nothing wrong in services sites!')
                                                              
def check_mom_bts(df_tw, tw_index, tw_col, df_msa,msa_index, msa_col):

    #c = country   
    msa_bts = df_msa[df_msa[msa_col]=='Yes']
    msa_bts_sites = [i for i in msa_bts[msa_index]]

    tw_bts = df_tw[df_tw[tw_col]=='Yes']
    tw_bts_sites = [i for i in tw_bts[tw_index]]

    out_tower_bts = [i for i in msa_bts_sites if i not in tw_bts_sites]
    filtered = tw_bts[tw_bts[tw_index].isin(out_tower_bts)]
    return filtered[[tw_index, tw_col]]         

def check_lc_ta_dates(df,tw_index, start_date,end_date):

        filtered = df[df[start_date] > df[end_date]]
        return filtered[[tw_index, start_date,end_date]]

def check_uip_tw(df_tw,tw_index, status_tw_col, decom_col, tw_bts_col, tw_critical_col, df_uip, uip_sites, country):
    t1 = ['pt', 'de', 'cz', 'ie', 'es', 'ro', 'hu']
    if country.lower() in t1:
        #tw_active = df_tw[df_tw['Site Status']=='In Service']
        count_tw_sites = [i for i in df_tw[df_tw[status_tw_col] =='In Service'][tw_index]]

        # check number of sites that are in uip file and doesn't have in df_tw
        in_service_uip_sites = []
        if not set(count_tw_sites).intersection(uip_sites):
            in_service_uip_sites = [i for i in uip_sites if i not in count_tw_sites]
        in_service_uip_sites = pd.DataFrame(in_service_uip_sites,columns=['Site In service out of UIP File!'])
        
        #check for decomissioned site not in uip files
        if decom_col != "":
            tw_decomiss = [i for i in df_tw[df_tw[decom_col]=='Yes'][tw_index] if i in uip_sites]
            decomiss_sites_in_uip = pd.DataFrame(tw_decomiss, columns=['Decomissioned Site in UIP File'])
        
            #Check BTS sites
            bts_sites = [i for i in df_tw[df_tw[tw_bts_col]=='Yes'][tw_index]]
            bts_sites_out_uip = []
            if not set(bts_sites).intersection(uip_sites):
                bts_sites_out_uip = [i for i in bts_sites if i not in uip_sites]
            bts_sites_out_uip = pd.DataFrame(bts_sites_out_uip, columns=['BTS Site not in UIP File'])

            #  Check for UIP critical sites 
            uip_critical = [i for i in df_uip[(df_uip['Commercials for sites beyond 10% cap of critical sites (Annual)']!='')&\
                                        df_uip['BTS site applicable charge (Annual)']!=""]['Site_ID']]
            bts_tw_critical = df_tw[df_tw[tw_critical_col]=='Beyond 10%'][tw_index]
            critical = []
            if len(uip_critical) > 0:
                if set(uip_critical).intersection(bts_tw_critical):
                    critical = [i for i in bts_tw_critical if i not in uip_critical]
            critical = pd.DataFrame(critical, columns=['Sites with critical level beyond 10% in out UIP File'])

        
            return in_service_uip_sites, decomiss_sites_in_uip, bts_sites_out_uip, critical
        else:
            #Check BTS sites
            bts_sites = [i for i in df_tw[df_tw[tw_bts_col]=='Yes'][tw_index]]
            bts_sites_out_uip = []
            if not set(bts_sites).intersection(uip_sites):
                bts_sites_out_uip = [i for i in bts_sites if i not in uip_sites]
            bts_sites_out_uip = pd.DataFrame(bts_sites_out_uip, columns=['BTS Site not in UIP File'])

            #  Check for UIP critical sites 
            uip_critical = [i for i in df_uip[(df_uip['Commercials for sites beyond 10% cap of critical sites (Annual)']!='')&\
                                        df_uip['BTS site applicable charge (Annual)']!=""]['Site_ID']]
            bts_tw_critical = df_tw[df_tw[tw_critical_col]=='Beyond 10%'][tw_index]
            critical = []
            if len(uip_critical) > 0:
                if set(uip_critical).intersection(bts_tw_critical):
                    critical = [i for i in bts_tw_critical if i not in uip_critical]
            critical = pd.DataFrame(critical, columns=['Sites with critical level beyond 10% in out UIP File'])
            return in_service_uip_sites, bts_sites_out_uip, critical
    else:
        #tw_active = df_tw[df_tw['Site Status']=='In Service']
        count_tw_sites = [i for i in df_tw[df_tw[status_tw_col] =='In Service'][tw_index]]

        # check number of sites that are in uip file and doesn't have in df_tw
        in_service_uip_sites = []
        if not set(count_tw_sites).intersection(uip_sites):
            in_service_uip_sites = [i for i in uip_sites if i not in count_tw_sites]
        in_service_uip_sites = pd.DataFrame(in_service_uip_sites,columns=['Site In service out of UIP File!'])
        
        #check for decomissioned site not in uip files
        tw_decomiss = [i for i in df_tw[df_tw[decom_col]=='Yes'][tw_index]]
        decomiss_sites_in_uip = []
        if set(tw_decomiss).intersection(uip_sites):
            decomiss_sites_in_uip = [i for i in tw_decomiss if i in uip_sites]
        decomiss_sites_in_uip = pd.DataFrame(decomiss_sites_in_uip, columns=['Decomissioned Site in UIP File'])
        
        #Check BTS sites
        bts_sites = [i for i in df_tw[df_tw[tw_bts_col]=='Yes'][tw_index]]
        bts_sites_out_uip = []
        if not set(bts_sites).intersection(uip_sites):
            bts_sites_out_uip = [i for i in bts_sites if i not in uip_sites]
        bts_sites_out_uip = pd.DataFrame(bts_sites_out_uip, columns=['BTS Site not in UIP File'])

        #  Check for UIP critical sites 
        uip = [i for i in df_uip['Site_ID']]
        bts_tw_critical = df_tw[df_tw[tw_critical_col]=='Yes'][tw_index]
        critical = []
        if set(uip).intersection(bts_tw_critical):
            critical = [i for i in bts_tw_critical if i not in uip]
        critical = pd.DataFrame(critical, columns=['Sites with critical level beyond 10% out in UIP File'])

        return in_service_uip_sites, decomiss_sites_in_uip, bts_sites_out_uip, critical

def check_commercial(path_current, path_before, col_replace, col_names, merge_cols, col_order):
    
    def check_uip_commercial_values(df_actual, df_before, merge_cols):
        df_atual = uip_comercial_actual
        df_ant = uip_comercial_before
        df_cross = pd.merge(df_actual, df_before, on=merge_cols, \
                            how='left', suffixes=('_actual', '_before'), indicator='Exist')
        df_cross['Exist'] = np.where(df_cross.Exist == 'both', 'Yes', 'No')
        df_cross['Equal Values'] = df_cross['Exist']
        
        return df_cross
    # Check for commercial Values into current UIP File and compare with UIP File before
    uip_comercial_actual = pd.read_excel(path_current,sheet_name='Commercial', names=col_names)
    #uip_comercial_actual = uip_comercial_actual[['Charge_Type', 'Data_Type', 'Input_Value']]
    uip_comercial_actual = replace_values(uip_comercial_actual, col_replace, value=0)

    uip_comercial_before = pd.read_excel(path_before,sheet_name='Commercial', names=col_names )
    #uip_comercial_before = uip_comercial_before[['Charge_Type', 'Data_Type', 'Input_Value']]
    uip_comercial_before = replace_values(uip_comercial_before, col_replace, value=0)

    df_commercial =  check_uip_commercial_values(uip_comercial_actual, uip_comercial_before, merge_cols)

    df_commercial = df_commercial.reindex(columns=col_order)
    df_commercial_diffs = df_commercial[df_commercial['Equal Values']=='No']
    return df_commercial_diffs

def general_log_erros(df_list, sheet_list, path):
    writer = pd.ExcelWriter(path,engine='openpyxl')   
    for dataframe, sheet in zip(df_list, sheet_list):
        dataframe.to_excel(writer, sheet_name=sheet, startrow=0 , startcol=0)   
    writer.save() 

def find_diffs_between_files(path_OLD, path_NEW, index_col, bill_cols, \
                             status_col, path_save, old_file, new_file, type_file='mix',kind='tw', sheetname='', skipr=0, skipc=0):
    
    def lower_str(columns):
        newlist = list(map(lambda x: x.lower(), columns))
        return newlist

    def fit_df(df, bill_cols):
        fit_cols = lower_str(list(df.columns))
        df.columns = fit_cols
        df = df[bill_cols]
        df = df.dropna(subset=[index_col], axis=0)
        df = df.drop_duplicates(subset=[index_col], keep='last')
        df[index_col] = df[index_col].astype(str)
        df['sites'] = df[index_col]
        # Aqui ajusta os nan e nat dos DFs, quando não forem arquivos não lidos
        df = df.set_index('sites').fillna('')
        return df
        
    def change_format(index, worksheet, format):
        for cell in worksheet[f"{str(index)}:{str(index)}"]:
            cell.font = format

    bill_cols = lower_str(bill_cols)
    index_col = index_col.lower()
    type_file = type_file.lower()
    status_col = status_col.lower()
    if type_file == 'excel':
        df_OLD = pd.read_excel(path_OLD,sheet_name = sheetname, skiprows = skipr).fillna('')
        df_OLD = df_OLD.iloc[:,skipc:]
        df_OLD = fit_df(df_OLD,bill_cols)

        df_NEW = pd.read_excel(path_NEW,sheet_name = sheetname, skiprows = skipr).fillna('')
        df_NEW = df_NEW.iloc[:,skipc:]
        df_NEW = fit_df(df_NEW,bill_cols)

    elif type_file == 'csv':
        df_OLD = pd.read_csv(path_OLD, encoding='latin').fillna('')
        df_OLD = fit_df(df_OLD,bill_cols)

        df_NEW = pd.read_csv(path_NEW, encoding='latin').fillna('')
        df_NEW = fit_df(df_NEW,bill_cols)

    elif type_file == 'mix':
        df_OLD = pd.read_csv(path_OLD, encoding='latin')
        df_OLD = fit_df(df_OLD,bill_cols)

        df_NEW = pd.read_excel(path_NEW,sheet_name = sheetname, skiprows = skipr)
        df_NEW = df_NEW.iloc[:,skipc:]
        df_NEW = fit_df(df_NEW,bill_cols)
    else: 
        df_OLD = fit_df(path_OLD,bill_cols)
        df_NEW = fit_df(path_NEW,bill_cols)


    # Perform Diff
    new_copy = df_NEW.copy()
    droppedRows = []
    newRows = []

    cols_OLD = df_OLD.columns
    cols_NEW = df_NEW.columns
    sharedCols = list(set(cols_OLD).intersection(cols_NEW))

    for row in new_copy.index:
        if (row in df_OLD.index) and (row in df_NEW.index):
            for col in sharedCols:
                value_OLD = df_OLD.loc[row,col]
                value_NEW = df_NEW.loc[row,col]
               #Error de a.empty() são sites duplcados e nã poodem estar no index
                if value_OLD == value_NEW:
                    new_copy.loc[row,col] = np.nan
                else:
                    new_copy.loc[row,col] = f'{value_OLD} > {value_NEW}'
        else:
            newRows.append(row)

    new_copy = new_copy.dropna(axis=0, how='all')
    new_copy = new_copy.dropna(axis=1, how='all')

    for row in df_OLD.index:
        if row not in df_NEW.index:
            droppedRows.append(row)
            new_copy = new_copy.append(df_OLD.loc[row,:])
    
    new_copy = new_copy.sort_index(key=lambda x: x.str.lower()).fillna('')
    
    new_copy = new_copy.reset_index()
    if kind=='tw':
        sites = [i for i in new_copy['sites']] 
        old = df_OLD[[status_col]].reset_index()
        old = old.loc[old['sites'].isin(sites)]
        new = df_NEW[[status_col]].reset_index()
        new = new.loc[new['sites'].isin(sites)]
        df_cross = pd.merge(new, old, on=['sites'], how='inner', suffixes=('_current', '_before'))
        new_copy = pd.merge(new_copy, df_cross, on=['sites'], how='left')


    print(f'\nNew Rows:     {newRows}')
    print(f'Dropped Rows: {droppedRows}')

    # Save output and format
    fname = f'{path_save} - ({old_file}) vs ({new_file}).xlsx'
    file = pd.ExcelWriter(fname, engine='openpyxl')
    
    new_copy.to_excel(file, sheet_name='diffs_founded', index=False)
    df_NEW.to_excel(file, sheet_name='new_file', index=False)
    df_OLD.to_excel(file, sheet_name='old_file', index=False)

    # get openpyxl objects
    wb  = file.book
    ws = file.sheets['diffs_founded']
    
    red_fill = PatternFill(start_color='95A7B3', \
                                end_color='95A7B3', fill_type='solid')
    red_font = Font(color='00FF0000', italic=True)
    new_fmt = Font(color='3F976D',bold=True, italic=True)
    removed = Font(color='95A7B3',bold=True, italic=True)

    dxf = DifferentialStyle(font=red_font, fill=red_fill)
    highlight = Rule(type="containsText", operator="containsText", text=">", dxf=dxf)
    highlight.formula = ['SEARCH(">", A1)']
    ws.conditional_formatting.add('A1:ZZ1000', highlight)
    
     
    #print(len(newRows))
    for site in new_copy['sites']:
        if site in newRows:
            idx = new_copy.index[new_copy[index_col]==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,new_fmt)
        if site in droppedRows:
            idx = new_copy.index[new_copy[index_col]==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,removed)
    
    wb.save(fname)
    print('\nDone.\n')

    

Looking for differences betwwnn older and newest files in towerdb.

In [ ]:
def find_diffs_between_files(path_OLD, path_NEW, index_col, bill_cols, path_save, old_name,new_name, type_file='mix', 
                             status_col='', kind='tw', dates=[], sheetname='', skipr=0, skipc=0):
    
    def lower_str(columns):
        newlist = list(map(lambda x: x.lower(), columns))
        return newlist

    def fit_df(df, index_col):
        #print(df.head(1))
        df = df.dropna(subset=[index_col], axis=0)
        df[index_col] = df[index_col].astype(str)
        df['sites'] = df[index_col]
        # Aqui ajusta os nan e nat dos DFs, quando não forem arquivos não lidos
        df = df.set_index('sites').fillna('')
        return df
        
    def change_format(index, worksheet, format):
        for cell in worksheet[f"{str(index)}:{str(index)}"]:
            cell.font = format

    def csv_header(path):
        """Função usada para ler os ficheiros que tem o simbolo do Euro"""
        import csv
        f = open(path, encoding='windows-1252', errors='ignore')
        data = []
        for row in csv.reader(f, delimiter=','):
            data.append(row)
        col = lower_str([*data[0]])
        #data.pop(0)
        #df = pd.DataFrame(data, columns=col)
        return  col

    def comparison(df_old, df_new,status):
        # Perform Diff
        new_copy = df_NEW.copy()
        droppedRows = []
        newRows = []

        cols_OLD = list(df_OLD.columns)
        cols_NEW = list(df_NEW.columns)
        sharedCols = list(set(cols_OLD).intersection(cols_NEW))
        #print(sharedCols)
        for row in new_copy.index:
            if (row in df_OLD.index) and (row in df_NEW.index):
                for col in sharedCols:
                    value_OLD = df_OLD.loc[row,col]
                    value_NEW = df_NEW.loc[row,col]
                #Error de a.empty() são sites duplcados e nã poodem estar no index
                    if value_OLD == value_NEW:
                        new_copy.loc[row,col] = np.nan
                    else:
                        new_copy.loc[row,col] = f'{value_OLD} > {value_NEW}'
            else:
                newRows.append(row)

        new_copy = new_copy.dropna(axis=0, how='all')
        new_copy = new_copy.dropna(axis=1, how='all')

        for row in df_OLD.index:
            if row not in df_NEW.index:
                droppedRows.append(row)
                new_copy = new_copy.append(df_OLD.loc[row,:])
        
        new_copy = new_copy.sort_index(key=lambda x: x.str.lower()).fillna('')
        
        new_copy = new_copy.reset_index()

        if kind=='tw':
            sites = [i for i in new_copy['sites']] 
            old = df_OLD[[status]].reset_index()
            old = old.loc[old['sites'].isin(sites)]
            new = df_NEW[[status]].reset_index()
            new = new.loc[new['sites'].isin(sites)]
            df_cross = pd.merge(new, old, on=['sites'], how='inner', suffixes=('_current', '_before'))
            new_copy = pd.merge(new_copy, df_cross, on=['sites'], how='left')
            status_1 = f'{status}_current'
            status_2 = f'{status}_before'
            new_copy = new_copy.set_index('sites')
            new_copy = new_copy[[status_1, status_2]+ new_copy.columns[:-2].tolist()]
            new_copy = new_copy.reset_index()

        return newRows, droppedRows, new_copy

    def date_parser(df, columns, format=1, type_dates='normal'):
        t_col = type_dates.lower()
        if format == 1:
            type_date = "%d/%m/%Y"
        else:
            type_date = "%d-%m-%Y"
        for column in lower_str(columns):
            if t_col == 'mixed':
                df[column] = [date_obj.strftime("%d/%m/%Y") if not pd.isnull(date_obj) \
                            and not isinstance(date_obj, str) else date_obj for date_obj in df[column]]
                df[column] = df[column].astype(str)
            else:
                df[column] = [date_obj.strftime("%d/%m/%Y") if not pd.isnull(date_obj) else '' for date_obj in df[column]]
                df[column] = df[column].astype(str)

    bill_cols = lower_str(bill_cols)
    index_col = index_col.lower()
    type_file = type_file.lower()
    status_col = status_col.lower()
    if type_file == 'excel':
        df_OLD = pd.read_excel(path_OLD,sheet_name = sheetname, skiprows = skipr).fillna('')
        df_OLD.columns = lower_str(list(df_OLD.columns)) 
        df_OLD = df_OLD.iloc[:,skipc:]
        df_OLD = fit_df(df_OLD,index_col,kind, kind_col)

        df_NEW = pd.read_excel(path_NEW,sheet_name = sheetname, skiprows = skipr).fillna('')
        df_NEW.columns = lower_str(list(df_NEW.columns)) 
        df_NEW = df_NEW.iloc[:,skipc:]
        df_NEW = fit_df(df_NEW,index_col,kind, kind_col)

    elif type_file == 'csv':
        #cols_old = csv_header(path_OLD)
        df_OLD = pd.read_csv(path_OLD, engine='python',encoding='latin1').fillna('')
        df_OLD = fit_df(df_OLD, index_col, kind, kind_col)

        cols_new = csv_header(path_NEW)
        #Se for o arquivo gerado a partir do xlsx não prcisa de encoding
        df_NEW = pd.read_csv(path_NEW,header=0, names=cols_new).fillna('')
        df_NEW = fit_df(df_NEW, index_col,kind, kind_col)

    else:
        #cols_old = csv_header(path_OLD)
        df_OLD = pd.read_csv(path_OLD,encoding='latin1').fillna('')
        cols_old = lower_str(list(df_OLD.columns))
        df_OLD.columns = cols_old
        df_OLD = fit_df(df_OLD, index_col)

        df_NEW = pd.read_excel(path_NEW,sheet_name = sheetname, skiprows = skipr)
        df_NEW.columns = lower_str(list(df_NEW.columns)) 
        df_NEW = df_NEW.iloc[:,skipc:]
        for i in dates:
            df_NEW[i] = [date_obj.strftime("%d/%m/%Y") if not pd.isnull(date_obj) \
                         and not isinstance(date_obj, str) else date_obj for date_obj in df_NEW[i]]
            df_NEW[i] = df_NEW[i].astype(str)
            
        df_NEW = df_NEW.dropna(subset=[index_col], axis=0)
        df_NEW[index_col] = df_NEW[index_col].astype(str)
        df_NEW['sites'] = df_NEW[index_col]
        # Aqui ajusta os nan e nat dos DFs, quando não forem arquivos não lidos
        df_NEW = df_NEW.set_index('sites').fillna('')
        #df_NEW = df_NEW[cols_old]

        #df_NEW = fit_df(df_NEW, index_col)

    newRows, droppedRows, df_all = comparison(df_OLD, df_NEW, status_col)

    print(f'\nNew Rows:     {newRows}')
    print(f'Dropped Rows: {droppedRows}')

    # Save output and format
    fname = f'{path_save} - ({old_name}) vs ({new_name}).xlsx'
    file = pd.ExcelWriter(fname, engine='openpyxl')
    
    df_all.to_excel(file, sheet_name='diffs_founded', index=False)
    df_NEW.to_excel(file, sheet_name='new_file', index=False)
    df_OLD.to_excel(file, sheet_name='old_file', index=False)

    # get openpyxl objects
    wb  = file.book
    ws = file.sheets['diffs_founded']
    
    red_fill = PatternFill(start_color='95A7B3', \
                                end_color='95A7B3', fill_type='solid')
    red_header = PatternFill(start_color='00FF0000', \
                                end_color='00FF0000', fill_type='solid')
    red_font = Font(color='00FF0000', italic=True)
    new_fmt = Font(color='3F976D',bold=True, italic=True)
    removed = Font(color='95A7B3',bold=True, italic=True)

    dxf = DifferentialStyle(font=red_font, fill=red_fill)
    highlight = Rule(type="containsText", operator="containsText", text=">", dxf=dxf)
    highlight.formula = ['SEARCH(">", A1)']
    ws.conditional_formatting.add('A1:ZZ10000', highlight)
    
    for column in bill_cols:
        dx = DifferentialStyle(font=Font(color='FFFFFF', bold=True), fill=red_header)
        header_style = Rule(type="containsText", operator="containsText", text=column, dxf=dx)
        header_style.formula = [f'SEARCH("{column}", A1)']
        ws.conditional_formatting.add('A1:ZZ10000', header_style)
    
    #print(len(newRows))
    for site in df_all['sites']:
        if site in newRows:
            idx = df_all.index[df_all[index_col]==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,new_fmt)
        if site in droppedRows:
            idx = df_all.index[df_all[index_col]==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,removed)

    wb.save(fname)
    print('\nDone.\n')

    
pathtw = '/content/TowerDB_Spain_20210809.xlsx'
pathold = '/content/TowerDB_Spain_20210731.csv'
to_save = '/content/TW_ES'
sheet= 'Enhanced TowerDB'
skipr = 7
skipc = 3
old_n = 'TWDB_20210731.csv'
new_n = 'TWDB_20210809.xlsx'
bill_cols = ['Code',\
             'Inhabitants',\
             'Categorization by Transmission Sys',\
             'Categorization by Site Type',\
             'Vodafone equipment giving Active Sharing to Orange',\
             'Bundled_Sites_Yes_No',\
			 'Wip_Site',\
			 'Region_For_Tax_Calculation',\
			 'Indoor_Outdoor_Categorization',\
			 'Bts_Site',\
			 'Sites_As_Metered_Estimated',\
			 'Indoor_Site_Any_Climate_Control',\
			 'Outdoor_Site_With__Power',\
			 'Bundled_Site_Categorization',\
			 'Strategic_Site',\
			 'Strategic_Site_Bucket',\
			 'Critical_Site',\
			 'CriticalSite_Beyond_10',\
			 'Active_Sharing_Arrangement',\
			 'Orange_Crossed_Site',\
			 'Das_Classification',\
			 'Macro_Core_Site_Yes_No',\
			 'Macro_Transmission_Hub_Yes_No',\
			 'Macro_Transmission_Hub_With_Shelters_Without_Shelters',\
			 'Transmission_With_Shelters_Without_Shelters',\
			 'Subsequent_Sharing_Arrangement',\
			 'First_Active_Sharing_Deployment_Type',\
             'First_Active_Sharing_Start_Date',\
             'First_Active_Sharing_End_Date',\
			 'Decommissioned_Site only for VF',\
			 'Sites_Fall_Under_2400',\
			 'Macro_CoreA_CoreB',\
			 'Billing Trigger date',
             ]

dates_tw = ['First_Active_Sharing_Start_Date','First_Active_Sharing_End_Date']
"""(path_OLD, path_NEW, index_col, bill_cols, path_save, old_name,new_name, type_file='mix', 
                             status_col='', kind='tw', dates=[], sheetname='', skipr=0, skipc=0)"""
find_diffs_between_files(pathold, pathtw, 'Code', bill_cols,to_save, old_n, new_n, type_file='mix',status_col='On air / Active', 
                         kind='tw',dates=dates_tw, sheetname=sheet, skipr=7, skipc=3)

Adding new columns for UIP

In [ ]:
# Lendo o Ficheiro de input
path_uip=''
sheet = 'SiteLevel'
site_level = pd.read_excel(path_uip,sheet, header=[0,1,2])

head_1 = pd.MultiIndex.from_product([[''],['Numeric (in months)'],['Delay in Site Modification Projects', 'Delay in BTS Projects']])
df_1 = pd.DataFrame(columns=head_1)

head_2 = pd.MultiIndex.from_product([[''],['Numeric (in €)'],['Excess of Upgrade Capital Expenditure over Threshold']])
df_2 = pd.DataFrame(columns=head_2)

site_level = pd.concat([site_level, df_1, df_2], axis=1)
site_level.head(1)

Doing all checks in Towerdb File

In [24]:
pathmsa = '/content/TowerDB_Spain_20210731.csv'
msa = pd.read_csv(pathmsa, encoding='latin')
msa.columns.to_list()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (112) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


['Code',
 'Site Name',
 'Macro Region',
 'Region',
 'Province',
 'Municipality',
 'Inhabitants',
 'Address',
 'Latitude',
 'Longitude',
 'Altitude',
 'Categorization by Transmission Sys',
 'Categorization by Transmission Sys_1',
 'Categorization by Transmission Sys (sub-cluster)',
 'MSA Type updated',
 'Categorization by Site Type',
 'Categorization by inhabitants',
 'Rural/ Suburban/ Urban',
 'Categorization by CONNECTIVITY as per MSA Site List 31/03',
 'POD ID',
 "Energy Consumption feb'20-jan´21",
 "Actual Energy cost feb'20-jan´21",
 'Infrastructure ready (existing)/ to be ready (new)',
 'Infrastructure to be shared by',
 'Counterpart',
 '# of Lease Contracts',
 'Current annual lease fees ',
 'Current energy lease fees ',
 'Current annual other fees ',
 'Total Annual Lease',
 '(Average) residual duration',
 'Maturity Cluster',
 'ExCo rep. Avg Annual Lease costs',
 'Total Energy Cost (Energy provider + LL)',
 'VOD (y/n)',
 'ORANGE',
 'Annual Fee per Tenant MNO1',
 'Annual Energy Fee

In [25]:
towerdb.columns.to_list()

['Code',
 'Site Name',
 'Macro Region',
 'Region',
 'Province',
 'Municipality',
 'Inhabitants',
 'Address',
 'Latitude',
 'Longitude',
 'Altitude',
 'Categorization by Transmission Sys',
 'Categorization by Transmission Sys.1',
 'Categorization by Transmission Sys (sub-cluster)',
 'MSA Type updated',
 'Categorization by Site Type',
 'Categorization by inhabitants',
 'Rural/ Suburban/ Urban',
 'Categorization by CONNECTIVITY as per MSA Site List 31/03',
 'POD ID',
 "Energy Consumption feb'20-jan´21",
 "Actual Energy cost feb'20-jan´21",
 'Infrastructure ready (existing)/ to be ready (new)',
 'Infrastructure to be shared by',
 'Counterpart',
 '# of Lease Contracts',
 'Current annual lease fees ',
 'Current energy lease fees ',
 'Current annual other fees ',
 'Total Annual Lease',
 '(Average) residual duration',
 'Maturity Cluster',
 'ExCo rep. Avg Annual Lease costs',
 'Total Energy Cost (Energy provider + LL)',
 'VOD (y/n)',
 'ORANGE',
 'Annual Fee per Tenant MNO1',
 'Annual Energy Fee

In [27]:
pathtw = '/content/TowerDB_Spain_20210809.xlsx'
sheet= 'Enhanced TowerDB'
skipr = 7
skipc = 3
towerdb = read_files(pathtw, sheet, skipr, skipc, 'Code')
towerdb = towerdb.rename(columns={'Categorization by Transmission Sys.1': 'Categorization by Transmission Sys_1'})
#towerdb.columns = lower_str(list(towerdb.columns))

"""Defining variables which is gonna be reusable in checks"""
tw_index = 'Code'
tw_doer = 'Date of equipment removal (from MAR´21)'
tw_status = 'on air / active'
tw_bts = 'Bts_Site'
tw_bill = 'Billing Trigger date'
tw_wip = 'Wip_Site'
tw_decom = 'Decommissioned_Site only for VF'
#w_amount = 'Lease Contract - Current annual lease fee'
tw_critical = 'CriticalSite_Beyond_10'

pathmsa = '/content/TowerDB_Spain_20210731.csv'
msa = pd.read_csv(pathmsa, encoding='latin')
#msa.columns = lower_str(list(msa.columns))
msa_index = 'Code'
msa_bts = 'Bts_Site'
msa_doer = 'Date of equipment removal (from MAR´21)'
msa_status = 'on air / active'
msa_bill = 'Billing Trigger date'
msa_wip = 'Wip_Site'
msa_decom = 'Decommissioned_Site only for VF'
#msa_amount = 'Lease Contract - Current annual lease fee'
msa_critical = 'CriticalSite_Beyond_10'

col_order = list(msa.columns)

towerdb = towerdb[col_order]
#towerdb = towerdb.fillna('')


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (112) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Check columns received looking for missing columns that is gonna be used in rating engine

In [ ]:
"""Check Columns Received"""
df_cols = check_columns_received(towerdb, col_order)
df_cols
#No columns missing

First Check - Dates Formats (dd/mm/YYYY) 

Columns: Date of equipment removal (from MAR´21)

In [18]:
towerdb[tw_bill]

KeyError: ignored

In [17]:
"""You need to convert all values in cols for string format to check"""
def date_parser(df, columns, format, type_dates):
    t_col = type_dates.lower()
    for column in columns:
        if t_col == 'mixed':
            df[column] = [date_obj.strftime(format) if not pd.isnull(date_obj) \
                        and not isinstance(date_obj, str) else date_obj for date_obj in df[column]]
            df[column] = df[column].astype(str)
        else:
            df[column] = [date_obj.strftime(format) if not pd.isnull(date_obj) else '' for date_obj in df[column]]
            df[column] = df[column].astype(str)

# Columns to functions
dates_bill = [tw_index, tw_bill]
dates_doer = [tw_index, tw_doer]
#Columns to parser
bill=[tw_bill]
doer=[tw_doer]

date_parser(towerdb, bill, "%d/%m/%Y", 'mixed')  #não precisar checkar o bill dos sites in service
date_parser(towerdb, doer, "%d/%m/%Y", 'mixed')

actives_1 = towerdb[towerdb['on air / active']=='In Service']
no_actives_1 = towerdb[~(towerdb[tw_status]=='In Service')]

#Checking columns for errors
actives_dates_errors = check_date_columns(actives_1, tw_index, dates_bill, 2) 
# Actives sites with blank billing trigger date
no_actives_dates_errors = check_date_columns(no_actives_1, tw_index, dates_doer, 2) 
# No Actives sites with blank Date of Equipament Removal

KeyError: ignored

In [15]:
#Checking columns for errors
actives_1 = towerdb[towerdb['on air / active']=='In Service']
no_actives_1 = towerdb[~(towerdb[tw_status]=='In Service')]

actives_dates_errors = check_date_columns(actives_1, tw_index, dates_bill, 2) 
# Actives sites with blank billing trigger date
no_actives_dates_errors = check_date_columns(no_actives_1, tw_index, dates_doer, 2) 
actives_dates_errors

NameError: ignored

Second Check - TW Amount value General (xxx.xx)

Column(s): ???

In [ ]:
amount_cols = [tw_index, """???"""]
df_amount_errors = check_amounts(actives, tw_index, amount_cols, 1)
#No one error

Thirth - Check Picklist values All sites

Do this check in all sites

Check the picklist for each case

In [ ]:
picklist_tw_general = {
    'Categorization by Transmission Sys' : ['Macro', 'Public DAS']
}
pick_col_general = ['Code', 'Categorization by Transmission Sys']

df_general_pick = check_picklist(towerdb, tw_index, pick_col_general, picklist_tw_general)
df_general_pick
# Tem sites com e sem a flag filtrar na planilha

Fourth Check - Remove "N/A", "0" or "-" values

In [ ]:
towerdb = replace_values(towerdb, col_order, value="")
actives = replace_values(actives, col_order, value="")

Fifth Check MoM Sites (BTS, decomissoned...)

In [ ]:
"""Falta Coluna de Flag Indicating BTS Site no twerdb recebido"""
actives = towerdb[towerdb[tw_status]=='In Service']
df_mom_bts = check_mom_bts(actives, tw_index, tw_bts, msa, msa_index, msa_bts)
# No one error df_mom_bts

decomiss = towerdb[towerdb[tw_status]=='Decommissioned']
df_mom_decom = check_mom_bts(decomiss, tw_index, tw_decom, msa, msa_index, msa_decom)
# No one error df_mom_decom

Check Picklist and dates formats for In service sites




In [ ]:
picklis_dict = {
    'Categorization by Transmission Sys': ['Macro', 'Public DAS'],
    'Categorization by Site Type': ['DAS passive','GBT','RTT'],
    'Sites_As_Metered_Estimated': ['Estimated Model','Metered Model'],
    'Infrastructure ready (existing)/ to be ready (new)': [''],
    'Indoor_Site_Any_Climate_Control': ['No','Yes; Indoor Air Conditioning','Yes; Indoor Air Conditioning and Free Air cooling / Free cooling units'],
    'Bts_Site': ['Yes', 'No'],
    'Strategic_Site': ['Yes', 'No'],
    'Strategic_Site_Bucket': ['Yes - first 460', ''],
    'Critical_Site': ['Yes', 'No'],
    'CriticalSite_Beyond_10': ['Within 10%','Non Critical'],
    'Wip_Site': ['Yes', 'No'],
    'Decommissioned_Site only for VF': ['Yes', 'No'],
    'First_Active_Sharing_Deployment_Type': ['']
}

pick_cols = ['Code','Categorization by Transmission Sys','Categorization by Site Type','Sites_As_Metered_Estimated',\
             'Infrastructure ready (existing)/ to be ready (new)','Indoor_Site_Any_Climate_Control','Bts_Site',\
             'Strategic_Site','Strategic_Site_Bucket','Critical_Site','CriticalSite_Beyond_10','Wip_Site',\
             'Decommissioned_Site only for VF','First_Active_Sharing_Deployment_Type' ]
actives = actives.fillna('')
df_in_service_picklist = check_picklist(actives, tw_index, pick_cols, picklis_dict)
# No errors

In [ ]:
#check dates in columns
start_dates = ['First_Active_Sharing_Start_Date']

date_parser(actives, start_dates, "%d/%m/%Y", 'no')
actives['First_Active_Sharing_End_Date'] = actives['First_Active_Sharing_End_Date'].fillna('')
in_service_cols = ['Code', 'First_Active_Sharing_Start_Date', 'First_Active_Sharing_End_Date']
df_in_service_dates = check_date_columns(actives, tw_index, in_service_cols, 2)
# tem errors em df_in_service_dates sites ativos

Fifth Check BTS Flagged(Billing Trigger and Commercial)





In [ ]:
status = 'Yes'
df_bts_flagged = check_tw_bill_doer(towerdb, tw_index,tw_bill, tw_bts, status, 'doer')
#No errors

In [ ]:
""" BTS sites"""
path_uip = '/content/UserInput_Spain_202106 v1.xlsx'
uip_names = ['Site_ID', 'BTS site applicable charge (Annual)', 'Commercials for sites beyond 10% cap of critical sites(Annual)']
uip = pd.read_excel(path_uip ,sheet_name='SiteLevel',usecols=[0,1,2],skiprows=2)
uip.columns = uip_names

msa_sites = [i for i in msa[msa_index]]
tw_sites = [i for i in towerdb[tw_index]]
uip_sites = [i for i in uip['Site_ID']]


In [ ]:
new_sites, bts_out_uip, df_bts_errors = check_new_sites(towerdb, tw_index, tw_bts, tw_bill, msa_sites, tw_sites, uip_sites)
# New sites = 132
# Error on BTS sites out of UIP File

Sixtith Check on air sites

In [ ]:
df_on_air = check_on_air_sites(actives, tw_index, pick_cols)
# NO Errors

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Check Wip SItes

In [ ]:
wip_out_tw_list, df_wip_and_bts_flagged = check_wip(towerdb,tw_index, tw_wip, tw_bts, msa, msa_index, msa_wip)
#No errors

Check Decomissioned sites

In [ ]:
df_decom_sites = check_decommissioned(towerdb, tw_index, tw_decom, tw_doer)
#No errors

Check Doer columns for in service sites

Should not to be in past or different of blank

In [ ]:
#COluna Doer tem valores fora do formato
df_doer = check_tw_bill_doer(actives, tw_index, tw_doer, tw_bts, 'Yes', 'doer')
#No errors

*Tenth* - Check UIP Towerdb matches

In [ ]:
in_service_uip_sites, decomiss_sites_in_uip, bts_sites_out_uip, critical = check_uip_tw(towerdb,tw_index, tw_status, \
                                                              tw_decom, tw_bts,tw_critical, \
                                                              uip, uip_sites, 'gr')
# in_service_uip_sites sites out UIP Files
# bts_sites_out_uip BTS sites out UIP FIles

Commercial

In [ ]:

path_before = '/content/UserInput_Spain_20210630.xlsx'
names = ['Charge_Type','Sub_Charge_Type', 'Param1','Param2', 'Data_Type', 'Input_Value',\
        'Description/Instruction', 'Frequency of Update']
merge_cols = ['Charge_Type','Sub_Charge_Type', 'Param1','Param2', 'Data_Type', \
              'Description/Instruction', 'Frequency of Update']
cols_ordered = ['Charge_Type','Sub_Charge_Type', 'Param1','Param2','Data_Type','Input_Value_actual',\
                'Input_Value_before','Equal Values','Description/Instruction', 'Frequency of Update']
df_commercial_diffs = check_commercial(path_uip, path_before, 'Input_Value', names, merge_cols, cols_ordered)
#NO errors

In [ ]:
df_list = [actives_dates_errors, no_actives_dates_errors,  bts_out_uip]
sheetnames = ['Actives Sites without Billin trigger Date', 'Not In Service without DOER','BTS Flagged in Towerdb out of UIP File']

path = '/content/towerdb_es_errors.xlsx'
general_log_erros(df_list, sheetnames, path)

/usr/local/lib/python3.7/dist-packages/openpyxl/workbook/child.py:102: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


Lc Input

In [ ]:
pathtw = '/content/TowerDB_Spain_20210708.xlsx'
sheet= 'LC_INPUT_SPAIN'
skipr = 0
skipc = 0
lc = pd.read_excel(pathtw, sheet)

lc_cols = ['Código', '   Importe anual']
lc['Importe anual'] = lc['   Importe anual'].astype(str)

#Python interpretou como float a coluna de Import Anual
df_lc_amount = check_amounts(lc, 'Código', lc_cols)
# No errors

df_lc_dates = check_lc_ta_dates(lc,'Código', 'Inicio','Fin vigen/')
#NO erros

In [ ]:
df_lc_amount

Looking for differences between newest and oldest LC file.


In [ ]:
pathtw = '/content/TowerDB_Spain_20210708.xlsx'
pathold = '/content/TowerDB_Spain_20210617 v2.xlsx'
to_save_lc = '/content/lc_'
sheet_lc= 'LC_INPUT_SPAIN'
skipr = 0
skipc = 0
lc_bill = ['Código', '   Importe anual','Inicio','Fin vigen/']
find_diffs_between_files(pathold, pathtw, 'Código', lc_bill,'',to_save_lc, 'excel','lc',sheet_lc)

TA OSP

In [ ]:
pathtw = '/content/TowerDB_Spain_20210708.xlsx'
sheet= 'TA_Input_Lease_OSP_Spain'
skipr = 0
skipc = 0
ta_bill = ['Código', 'Importe anual','Inicio','F/cie/tec/']
ta_osp = pd.read_excel(pathtw, sheet)

ta_osp = ta_osp.fillna('')
ta_osp = ta_osp[ta_osp['Inicio']!=""]


ta_osp_cols = ['Código', 'Importe anual']  

ta_osp['Importe anual'] = ta_osp['Importe anual'].astype(str)

df_ta_osp_amount = check_amounts(ta_osp, 'Código', ta_osp_cols)
# No errors

df_ta_osp_dates = check_lc_ta_dates(ta_osp,'Código', 'Inicio','F/cie/tec/')
#NO erros


In [ ]:
pathtw = '/content/TowerDB_Spain_20210708.xlsx'
sheet= 'TA_Input_Lease_TME_Spain'
skipr = 0
skipc = 0
ta_tme = ta_tme.fillna('')
ta_tme = ta_tme[ta_tme['Inicio']!=""]
ta_tme = pd.read_excel(pathtw, sheet)


ta_tme_cols = ['Código', 'Importe anual']  

ta_tme['Importe anual'] = ta_tme['Importe anual'].astype(str)
df_ta_tme_amount = check_amounts(ta_tme, 'Código', ta_tme_cols)
# No errors

df_ta_tme_dates = check_lc_ta_dates(ta_tme,'Código', 'Inicio','F/cie/tec/')
#NO erros

In [ ]:
pathtw = '/content/TowerDB_Spain_20210708.xlsx'
sheet= 'TA_Input_Lease_MM_Spain'
skipr = 0
skipc = 0
ta_mm = pd.read_excel(pathtw, sheet)

ta_mm = ta_mm.fillna('')
ta_mm = ta_mm[ta_mm['Inicio']!=""]

ta_mm_cols = ['Código', 'Importe anual']  

ta_mm['Importe anual'] = ta_mm['Importe anual'].astype(str)
df_ta_mm_amount = check_amounts(ta_mm, 'Código', ta_mm_cols)
# No errors

df_ta_mm_dates = check_lc_ta_dates(ta_mm,'Código', 'Inicio','F/cie/tec/')
#NO erros

In [ ]:
pathtw = '/content/TowerDB_Spain_20210708.xlsx'
sheet= 'TA_Input_Lease_Others_Spain'
skipr = 0
skipc = 0
ta_others = pd.read_excel(pathtw, sheet)

ta_others = ta_others.fillna('')
ta_others = ta_others[ta_others['Inicio']!=""]

ta_o_cols = ['Código', 'Importe anual']

ta_others['Importe anual'] = ta_others['Importe anual'].astype(str)
df_ta_o_amount = check_amounts(ta_others, 'Código', ta_o_cols)
# No errors

df_ta_o_dates = check_lc_ta_dates(ta_others,'Código', 'Inicio','F/cie/tec/')
#NO erros

Looking for differences between newest and oldest TA files.

In [ ]:
sheet_osp = 'TA_Input_Lease_OSP_Spain'
to_save_osp = '/content/ta_osp_'
sheet_tme = 'TA_Input_Lease_TME_Spain'
to_save_tme = '/content/ta_tme_'
sheet_mm = 'TA_Input_Lease_MM_Spain'
to_save_mm = '/content/ta_mm_'
sheet_others = 'TA_Input_Lease_Others_Spain'
to_save_others = '/content/ta_others_'
skipr = 0
skipc = 0
ta_bill = ['Código', 'Importe anual','Inicio','F/cie/tec/']
find_diffs_between_files(pathold, pathtw, 'Código', ta_bill,'', to_save_osp, 'excel','ta',sheet_osp)
find_diffs_between_files(pathold, pathtw, 'Código', ta_bill, to_save_tme, 'excel',sheet_tme)
find_diffs_between_files(pathold, pathtw, 'Código', ta_bill, to_save_mm, 'excel',sheet_mm)
find_diffs_between_files(pathold, pathtw, 'Código', ta_bill, to_save_others, 'excel',sheet_others)